In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as pq


In [ ]:
#read parquet file /home/anjah/Documents/mag/BD/project/BD_project/data/augmented/augmented/2014_augmented.parquet and display properties
# Open the Parquet file
parquet_file = pq.ParquetFile('/home/anjah/Documents/mag/BD/project/BD_project/data/augmented/augmented/2014_augmented.parquet')

# Get basic file metadata
metadata = parquet_file.metadata
print("Number of Row Groups:", metadata.num_row_groups)
print("Number of Rows:", metadata.num_rows)
print("Number of Columns:", metadata.num_columns)
print("File Schema:\n", parquet_file.schema)
print("Column Names:", parquet_file.schema.names)

# Display row group information
for i in range(metadata.num_row_groups):
    row_group = metadata.row_group(i)
    print(f"\nRow Group {i}:")
    print("Number of Rows:", row_group.num_rows)
    for j in range(row_group.num_columns):
        col = row_group.column(j)
        print(f"  Column {j}: {col.path_in_schema}, Size: {col.total_compressed_size} bytes")

#1048576

In [ ]:
import h5py

def explore_hdf5(file_path):
    # Open the HDF5 file
    with h5py.File(file_path, 'r') as hdf:
        # Recursively explore the file structure
        def print_attrs(name, obj):
            print(f"{name}:")
            for key, val in obj.attrs.items():
                print(f"  Attribute: {key} = {val}")
            if isinstance(obj, h5py.Dataset):
                print(f"  Dataset shape: {obj.shape}")
                print(f"  Dataset dtype: {obj.dtype}")
            if isinstance(obj, h5py.Group):
                print(f"  Group: {name}")

        # Visit all groups and datasets in the file
        hdf.visititems(print_attrs)

# Example usage
explore_hdf5('/home/anjah/Documents/mag/BD/project/BD_project/data/augmented/augmented/hdf5/2014_augmented.h5')


In [30]:
import os
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
from datetime import datetime

cluster = LocalCluster(n_workers=1, threads_per_worker=4, memory_limit="8GB")
client = Client(cluster)


client

/home/anjah/miniconda3/envs/BDproj/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44607 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44607/status,
Dashboard: http://127.0.0.1:44607/status,Workers: 1
Total threads: 4,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41127,Workers: 1
Dashboard: http://127.0.0.1:44607/status,Total threads: 4
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:45609,Total threads: 4
Dashboard: http://127.0.0.1:38701/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:37633,


In [ ]:
hdf5_path = "/home/anjah/Documents/mag/BD/project/BD_project/data/augmented/augmented/hdf5/2014_augmented.h5"

    # Initialize Dask DataFrame
df = dd.read_hdf(hdf5_path, key='df')

In [ ]:
df.head()

In [ ]:
top_10_states = df['Registration State'].compute()

In [19]:
df = dd.read_parquet(os.path.join('/home/anjah/Documents/mag/BD/project/BD_project/data/augmented/augmented/', "*.parquet"))

In [26]:
start = datetime.now()
print(start)
#ram usage for computation
#ram_usage = df.memory_usage(deep=True).sum().compute()
#ram needed for the following computation
top_10_states = df['Registration State'].value_counts().nlargest(10).compute()
top_10_states_list = top_10_states.index.tolist()
    # Filter the Dask DataFrame for the top 10 states
df_top_10_dask = df[df['Registration State'].isin(top_10_states_list)]
end = datetime.now()
time = end - start
print(time)

2024-08-18 22:29:28.839524
0:00:00.506243


In [20]:
start = datetime.now()
print(start)
#ram usage for computation
#ram_usage = df.memory_usage(deep=True).sum().compute()
#ram needed for the following computation
top_10_states = df['Registration State'].value_counts().nlargest(10).compute()
top_10_states_list = top_10_states.index.tolist()
    # Filter the Dask DataFrame for the top 10 states
df_top_10_dask = df[df['Registration State'].isin(top_10_states_list)]
end = datetime.now()
time = end - start
print(time)

2024-08-18 22:47:56.773580
0:00:02.176026


In [21]:

#Aggregate the data to get the number of tickets per day

#Aggregate the data to get the number of tickets per day

#Aggregate the data to get the number of tickets per day
tickets_per_day = df.groupby('Issue Date').size().reset_index()


In [22]:
tickets_per_day.head(10)

2024-08-18 22:48:16,118 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.66 GiB -- Worker memory limit: 3.73 GiB


,Issue Date,0
0,2013-07-01,128
1,2013-07-02,272
2,2013-07-03,304
3,2013-07-04,117
4,2013-07-05,255
5,2013-07-06,189
6,2013-07-07,344
7,2013-07-08,484
8,2013-07-09,739
9,2013-07-10,652


In [23]:
tickets_per_day.columns = ['Issue Date', 'Number of Tickets']

In [24]:
sorted_tickets_per_day = tickets_per_day.sort_values(by='Number of Tickets', ascending=False)

In [25]:
# Assuming the DataFrame already contains the average precipitation per day
avg_precipitation_per_day = df[['Issue Date', 'PRCP']].drop_duplicates()

In [26]:
merged_data = sorted_tickets_per_day.merge(avg_precipitation_per_day, on='Issue Date')

In [31]:
start = datetime.now()
print(start)
merged_data.compute()
end = datetime.now()
time = end - start
print(time)

2024-08-18 22:49:50.534211
0:00:07.114756
